In [28]:
#Required Libraries
# !pip install graphviz
# !pip install seaborn
# !pip install pydotplus
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.feature_selection import f_classif, SelectPercentile

import sklearn
from scipy import stats
from sklearn.svm import SVC
from sklearn import  datasets
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,plot_confusion_matrix,accuracy_score
from mlxtend.plotting import plot_decision_regions
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz 
import seaborn as sns
from IPython.display import Image  
import pydotplus
from sklearn.naive_bayes import GaussianNB
from scipy.stats import norm
import pandas
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
import pandas
from sklearn.utils import shuffle
from sklearn import svm
# import xgboost as xgb
import imblearn
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
# from xgboost import XGBClassifier
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import f1_score
from pandas.api.types import CategoricalDtype 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import learning_curve,GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, f_classif


In [99]:
np.random.seed(50)
df = pandas.read_csv('KDDTrain_Initial.csv')
df_sub = pandas.read_csv('KDDTest_Initial.csv')
df["attack"][df["attack"] == "normal"] = 0;     df["attack"][df["attack"] == "attack"] = 1
df_sub["attack"][df_sub["attack"] == "normal"] = 0; df_sub["attack"][df_sub["attack"] == "attack"] = 1

C:\Users\Mohamed\anaconda3\envs\main\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Mohamed\anaconda3\envs\main\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
class Preprocessor( BaseEstimator, TransformerMixin ):
    def __init__( self, 
                 cols_tobe_dropped=[],
                 log_vars = [],
                 z_vars = [],
                 minmax_vars = [],
                 bin_vars = [],
                 catunique_vars=[],
                 labelencode_vars=[],
                 onehot_vars=[],
                 lowvariance_thr=1,
                 lowvariance_ignore=[],
                 percentile=10
                ):
        '''
        Params
        ---------------
        outlier_vars: list
            list of columns to be considered for removal when applying transform
        '''
        
        # Variables to be dropped
        self.cols_tobe_dropped = cols_tobe_dropped
        
        # Variables to scale down using log function
        self.log_vars = [c for c in log_vars if c not in self.cols_tobe_dropped]

        # Variables to apply z normalization on
        self.z_vars = [c for c in z_vars if c not in self.cols_tobe_dropped]

        # Variables to apply min max normalization on
        self.minmax_vars = [c for c in minmax_vars if c not in self.cols_tobe_dropped]

        self.percentile = percentile
        
        # Variables to apply binning on
        self.bin_vars = [c for c in bin_vars if c not in self.cols_tobe_dropped]

        # Variables to apply categorization by unique value on
        self.catunique_vars = [*catunique_vars, *bin_vars]
        self.catunique_vars = [c for c in self.catunique_vars if c not in self.cols_tobe_dropped]

        # Variables to one hot encode
        self.onehot_vars = [*onehot_vars]
        self.onehot_vars = [c for c in self.onehot_vars if c not in self.cols_tobe_dropped]
        
        # Variables to one label encode
        self.labelencode_vars = [c for c in labelencode_vars if c not in self.cols_tobe_dropped]

        # Vaiables that must be treated as categorical from the time it entered
        self.object_vars = self.labelencode_vars + self.onehot_vars
        self.object_vars = [v for v in self.object_vars if v not in \
                            self.catunique_vars + self.log_vars + self.z_vars + self.minmax_vars]
        
        self.lowvariance_thr = lowvariance_thr
        
        self.lowvariance_ignore = lowvariance_ignore
        
    def fit( self, df, outlier_removal=False, target_col=None, target_value=None ):
        self.target_col = target_col
        self.target_value = target_value
        
        df_clean = df.copy()
        
        # drop columns
        df_clean = df.drop(self.cols_tobe_dropped, axis=1)

        # Apply Nan and non-finite values removal so that fitting would occur on clean data
        df_clean = self.clean_nans(df_clean)
        
        # Change categorical variable columns to object so that categorical outlier removal may apply
        for col in self.object_vars:
            df_clean[col] = df_clean[col].astype(object)
        for col in [c for c in df_clean.columns if c not in self.object_vars]:
            df_clean[col] = pd.to_numeric(df_clean[col])
                
        # Apply outlier removal so that fitting would occur on clean data
        # perhaps outlier removal should follow log transformation
        if outlier_removal:
            df_clean = self.remove_outlier(df_clean)
        
        # Apply log2 transformation
        self._min_log = {} # the minimum value for each log var
        for v in self.log_vars:
            self._min_log[v] = min(df_clean[v])
            df_clean[v] = np.log2([v if v > 1 else 1 for v in (df_clean[v].values - min(df_clean[v]) + 1)])
        
        # Apply z normalization
        self._trans_z = {} # the standard scaler transformers for each z var
        for z in self.z_vars:
            scaler = StandardScaler()
            df_clean[z] = scaler.fit_transform(np.reshape(df_clean[z].values, (-1,1)))
            self._trans_z[z]=scaler

        # Apply min max normalization
        self._trans_minmax = {} # the standard scaler transformers for each z var
        for mm in self.minmax_vars:
            scaler = MinMaxScaler()
            df_clean[mm] = scaler.fit_transform(np.reshape(df_clean[mm].values, (-1,1)))
            self._trans_minmax[mm]=scaler

        # Apply feature selection
        str_df = df_clean.select_dtypes(include='object')
        num_df = df_clean.select_dtypes(include='number')
        self.selected_columns = df_clean.columns
        if self.target_col and self.percentile:
            self.sel = SelectPercentile(f_classif, percentile=self.percentile).\
            fit( num_df.loc[:, num_df.columns != self.target_col], df_clean[self.target_col])
            self.selected_columns = [*self.sel.get_feature_names_out(), *str_df.columns]
            if self.target_col not in self.selected_columns: self.selected_columns.append(self.target_col)
            df_clean = df_clean[self.selected_columns]
            
            # Readjust cat unieuq, label encode and one-hot encoding vars
            self.labelencode_vars = [c for c in self.labelencode_vars if c in self.selected_columns]
            self.onehot_vars = [c for c in self.onehot_vars if c in self.selected_columns]
            self.catunique_vars = [c for c in self.catunique_vars if c in self.selected_columns]
            self.bin_vars = [c for c in self.bin_vars if c in self.selected_columns]

            
        # Apply binning ===============================================
        self._bins = {} # the bins used for each bin var
        for b in self.bin_vars:
            _, bins = pd.qcut(df_clean[b].copy(), q=6, duplicates='drop', retbins=True)
            # expand the bins boundaries to infinity
            bins[0] = -np.inf
            bins[-1] = np.inf
            self._bins[b]=bins
            
            v = pd.cut(df_clean[b], bins, duplicates='drop')
            df_clean[b] = v.astype(str)
            
        # Apply categorization by unique value
        self._cat_unique = {} # unique values for each cat unique vars
        for cc in self.catunique_vars:
            unique_values = np.unique(df_clean[cc])
            freq_unique_values = []
            
            # Only take the values that are frequent (repeated more than 5% of the data)
            for u in unique_values:
                if sum(df_clean[cc] == u) > 0.05*len(df_clean):
                    # we need to put the values here so that the categorization transformer would work
                    df_clean.loc[df_clean[cc] == u, cc]  = str(u) 
                    freq_unique_values.append(u)
        
            # Set any value that was not in the transformation set to 'other'
            df_clean.loc[df_clean.applymap(np.isreal)[cc],cc] = "other" # same here
            self._cat_unique[cc]=freq_unique_values
        
        # Apply low variance removal on columns with dominant values
#         df_var = df_clean.copy()
#         df_var = df_var.loc[:, [c for c in df_clean.columns if c not in self.lowvariance_ignore]]
#         string_features = df_var.select_dtypes(include='object')
#         for str_fea in string_features.columns:
#             df_var[str_fea] = LabelEncoder().fit_transform(df_var[str_fea])
#         self.sel = VarianceThreshold(threshold=(self.lowvariance_thr))
#         self.sel = self.sel.fit(df_var)
        
#         df_clean = df_clean[[*self.sel.get_feature_names_out(), *self.lowvariance_ignore]]
        
        # Apply label Encoding for specified categorical variables
        self._le_cat = {} # label encoders for each specified column
        for le_var in self.labelencode_vars:
            le = LabelEncoder().fit(df_clean[le_var])
            self._le_cat[le_var]=le
        
        # Apply One-hot Encoding for categorical variables
#         self._trans_cat = None
#         if len(self.onehot_vars) > 0:
        cat_indices = [df_clean.columns.get_loc(col_name) for col_name in self.onehot_vars]
        self._trans_cat = ColumnTransformer(transformers=[('cat', OneHotEncoder(), cat_indices)], remainder='drop', sparse_threshold=0)
        self._trans_cat.fit(df_clean) 
        return self

    def transform( self, df, outlier_removal=False ):
        '''
        Params
        ---------------
        outlier_removal: str -- Changed
            strategy for outlier removal, default is None resulting in no outlier removal
            possible string values:
            "before" : apply outlier removal before preprocessing
            "after" : apply outlier removal after preprocessing
        '''
        df_clean = df.copy()
        
        # Drop columns
        df_clean = df.drop(self.cols_tobe_dropped, axis=1).reset_index().iloc[:,1:]
        
        # Apply Nan and non-finite values removal
        df_clean = self.clean_nans(df_clean)

        # Change categorical variable columns to object so that categorical outlier removal may apply
        for col in self.labelencode_vars + self.onehot_vars:
            df_clean[col] = df_clean[col].astype(object)
        
        # Apply log2 transformation
        for v, m in self._min_log.items():
            df_clean[v] = np.log2([v if v > 1 else 1 for v in (df_clean[v].values - m + 1)])

        # Apply z normalization
        for z, trans in self._trans_z.items():
            df_clean[z] = trans.transform(np.reshape(df_clean[z].values, (-1,1)))

        # Apply min max normalization
        for mm, trans in self._trans_minmax.items():
            df_clean[mm] = trans.transform(np.reshape(df_clean[mm].values, (-1,1)))

        # Apply low variance removal on columns with dominant values
        df_clean = df_clean[self.selected_columns]
            
        # Apply outlier removal before preprocessing
        if outlier_removal:
            df_clean = self.remove_outlier(df_clean)
            
        # Apply binning ===============================================
        for b, bins in self._bins.items():
            v = pd.cut(df_clean[b], bins, duplicates='drop')
            df_clean[b] = v.astype(str)

        # Apply categorization by unique value
        for cc, unique_values in self._cat_unique.items():
            # Set "other' for unkown unique values 
            for u in unique_values:
                df_clean.loc[df_clean[cc] == u, cc]  = str(u) 
            # Set any value that was not in the transformation set to 'other'
            df_clean.loc[df_clean.applymap(np.isreal)[cc],cc] = "other"
        
        # Contains one hot encoded variables + dataframe variables
        for le_var, le in self._le_cat.items():
            df_clean[le_var] = le.transform(df_clean[le_var])
        
        # Apply One-hot Encoding for categorical variables
        cat_indices = [df_clean.columns.get_loc(col_name) for col_name in self.onehot_vars]
        
        # Contains one hot encoded variables + dataframe variables
        x = np.array(self._trans_cat.transform(df_clean))
        
        # get the columns names for the categorical data columns only
        feature_names = [[f"{c}_{u}" for u in np.unique(df_clean[c])] for c in self.onehot_vars]
        feature_names = [y for x in feature_names for y in x]

        # drop categorical columns from the real dataframe
        df_clean = df_clean.drop(self.onehot_vars, axis=1)

        # create the categorical data frame (contains the categorical colummns)
        df_cat = pd.DataFrame(x[:,:len(feature_names)], columns=feature_names).astype(float)
        # concatenate the dataframe that contains no categorical columns and the one with categorical columns
        df_clean = pd.concat([df_clean, df_cat], axis=1)
        
        return df_clean
    
    
    def fit_transform(self, df, outlier_removal=False, target_col=None, target_value=None ):
        self.fit(df, outlier_removal=outlier_removal, target_col=target_col, target_value=target_value )
        return self.transform(df, outlier_removal=outlier_removal)
    
    
    def remove_outlier(self, df):
        df = df.copy()
        
        # remove numeric outliers
        num_df = df.select_dtypes(include='number')
        
        
#         print(len(df))
        if self.target_col and self.target_value is not None:
            df = df[~((np.abs(stats.zscore(num_df)) > 3).any(1) & np.array(df[self.target_col]==self.target_value))]
        else:
            df = df[~((np.abs(stats.zscore(num_df)) > 3).any(1))]
            
#         print(len(df))
#         for col in ['flag']:
#             print("\n"+col)
#             print(np.unique(df[col], return_counts=True))
        # remove string (categorical) outliers that are less than 5% of the data
        v = df.select_dtypes(include='object')
        if self.target_col is not None and self.target_value is not None:
            df = df[~(v.replace(v.stack().value_counts()).lt(0.05*len(df)).any(1)& np.array(df[self.target_col]==self.target_value))]
        else:
            df = df[~(v.replace(v.stack().value_counts()).lt(0.05*len(df)).any(1))]
#         print(len(df))
#         for col in ['flag']:
#             print("\n"+col)
#             print(np.unique(df[col], return_counts=True))
        return df.reset_index().iloc[:,1:]
    
    def clean_nans(self, df):
        # Remove NaNs
        df.dropna(inplace=True)
        df = df.reset_index().iloc[:,1:]
        
        # Remove non finit values
        num_df = df.select_dtypes(include='number')
        indices = np.unique(np.array(np.where(~(np.isfinite(num_df))))[0])
        df.drop(indices, inplace=True)
        df = df.reset_index().iloc[:,1:]
        
        return df

# df_pre = preprocessor.transform(df.iloc[:10000,:], outlier_removal=1) 

In [ ]:
cols_tobe_dropped = [
                    'urgent',
                    'land',
                    'duration',
                    'hot',
                    'num_failed_logins',
                    'wrong_fragment',
                    'num_file_creations',
                    'root_shell',
                    "num_compromised",
                    'su_attempted',
                    'num_root',
                    'num_shells',
                    'num_outbound_cmds',
                    'is_host_login',
                    'is_guest_login',
#                     'flag',
#                     'service',
#                     'protocol_type',
                    'dst_host_serror_rate',
                    'dst_host_srv_serror_rate',
                    'dst_host_rerror_rate',
                    'dst_host_srv_rerror_rate',
                    'num_access_files',
#                     'src_bytes',
#                     'dst_bytes', 
                    'srv_serror_rate',
                    'srv_rerror_rate',
                    'serror_rate',
                    'srv_diff_host_rate',
#                     'dst_host_same_srv_rate',
#                     'logged_in',
                    'rerror_rate',
                    'dst_host_srv_diff_host_rate',
#                     'level',
#                     'count',
                    ]

# Variables to scale down using log function
log_vars = ['src_bytes', 'dst_bytes', 'count', "srv_count"]
log_vars = [c for c in log_vars if c not in cols_tobe_dropped]

# Variables to apply z normalization on
z_vars = ['src_bytes', 'dst_bytes', 'count', 'srv_count', "wrong_fragment"]
z_vars = [c for c in z_vars if c not in cols_tobe_dropped]

# Variables to apply min max normalization on
minmax_vars = ['dst_host_count', 'dst_host_srv_count']
minmax_vars = [c for c in minmax_vars if c not in cols_tobe_dropped]

# Variables to apply binning on
bin_vars = ['same_srv_rate', 'diff_srv_rate']
bin_vars = [c for c in bin_vars if c not in cols_tobe_dropped]

# Variables to apply categorization by unique value on
catunique_vars = ['dst_host_srv_diff_host_rate', *bin_vars]
catunique_vars = [c for c in catunique_vars if c not in cols_tobe_dropped]

# Variables to one hot encode
labelencode_vars = ["protocol_type", "service", "flag", *catunique_vars]
labelencode_vars = [c for c in labelencode_vars if c not in cols_tobe_dropped]

# Replace this one value "tftp_u" in the test data
df_clean_sub = df_sub.copy()
indices = np.where(df_clean_sub["service"] == "ntp_u")
df_clean_sub["service"][indices[0]] = df_clean_sub["service"][0] 
indices = np.where(df_clean_sub["service"] == "tftp_u")
df_clean_sub["service"][indices[0]] = df_clean_sub["service"][0] 


preprocessor = Preprocessor(
    cols_tobe_dropped=cols_tobe_dropped,
    log_vars = log_vars,
    z_vars = z_vars,
    minmax_vars = minmax_vars,
    bin_vars = bin_vars,
    catunique_vars=catunique_vars,
    labelencode_vars=labelencode_vars,
    onehot_vars=[],
    percentile=0)

preprocessor.fit(df.iloc[:,:], outlier_removal=1, target_col='attack', target_value=0)

df_pre = preprocessor.transform(df.iloc[:,:], outlier_removal=1)
df_sub_pre = preprocessor.transform(df_clean_sub)

X = df_pre.loc[:, df_pre.columns != "attack"]
y = df_pre["attack"]
X_sub  = df_sub_pre.loc[:, df_sub_pre.columns != "attack"]
y_sub = df_sub_pre["attack"]

# X = np.array(x_train)
# y = np.array(y_train)
# X_sub = np.array(X_sub)
# y_sub = np.array(y_sub)

In [176]:
# Split and save tthe data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=1, stratify=y)

X_train = X_train.assign(e=pd.Series(y_train).values)
X_train.rename({'e': 'attack'}, axis=1, inplace=True)
print(len(X_train))
X_train.drop_duplicates()
print(len(X_train))
X_train.to_csv('./Train.csv', index=False)

X_val = X_val.assign(e=pd.Series(y_val).values)
X_val.rename({'e': 'attack'}, axis=1, inplace=True)
print(len(X_val))
X_val.drop_duplicates()
print(len(X_val))
X_val.to_csv('./NoramTest.csv', index=False)

X_sub = X_sub.assign(e=pd.Series(y_sub).values)
X_sub.rename({'e': 'attack'}, axis=1, inplace=True)
print(len(X_sub))
X_sub.drop_duplicates()
print(len(X_sub))
X_sub.to_csv('./SpecificTest.csv', index=False)

92139
92139
23035
23035
22544
22544


In [177]:
for col in df_pre.columns:
    print("\n"+col)
    print(np.unique(df_pre[col], return_counts=True))


protocol_type
(array([0, 1, 2]), array([ 6982, 97564, 10628], dtype=int64))

service
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]), array([    1,     6,   862,     2,   719,   710,   734,   545,   563,
         521,   538,   531,  7441,   434,  4089,  2887,   485,   474,
        1222,   836,  6198,   518,     2,   460, 39939,     1,   530,
           2,   644,   687,   433,   299,   410,   475,   429,   439,
         451,   405,   347,   362,   360,   630,   296,  1755,     5,
          78,    78,    69, 21509,    78,    86,    61,  6786,   245,
         306,   381,   544,   477,  1436,     3,   578,     3,   780,
         689,   617,   693], dtype=int64))

flag
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([   35, 11152,  1343